In [ ]:
!pip install groq pandas plotly statsmodels ta matplotlib seaborn


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.3 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=0e9fccf2400a22e3fd33348929835269fa80dffb938c1a858055e5d2fb892fa3
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [ ]:
!python check_key.py
!python chatbot.py

Traceback (most recent call last):
  File "/content/check_key.py", line 3, in <module>
    from config.config import config
ModuleNotFoundError: No module named 'config'
python3: can't open file '/content/chatbot.py': [Errno 2] No such file or directory


In [ ]:
stock_forecast_code = """
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from statsmodels.tsa.arima.model import ARIMA
from ta.momentum import RSIIndicator
from ta.trend import SMAIndicator
import warnings
import sqlite3

warnings.filterwarnings("ignore")

# Native currency mapping for stocks
STOCK_CURRENCY = {
    'AAPL': 'USD', 'MSFT': 'USD', 'GOOGL': 'USD', 'TSLA': 'USD', 'NVDA': 'USD',
    'RELIANCE.NS': 'INR', 'TCS.NS': 'INR', 'INFY.NS': 'INR', 'HDFCBANK.NS': 'INR'
}

def load_data_from_db(db_path, symbol):
    \"\"\"Loads historical stock data from the SQLite database for a given symbol.\"\"\"
    with sqlite3.connect(db_path) as conn:
        query = f"SELECT date, open, high, low, close FROM market_data WHERE symbol = ? ORDER BY date"
        df = pd.read_sql_query(query, conn, params=(symbol,), index_col='date', parse_dates=['date'])
        if df.empty:
            raise ValueError(f"No data found for symbol '{symbol}' in the database at {db_path}.")
        return df

def get_fx_rate_from_db(db_path, from_currency, to_currency):
    \"\"\"Fetches the latest FX rate from the database.\"\"\"
    if from_currency == to_currency:
        return 1.0
    try:
        with sqlite3.connect(db_path) as conn:
            query = "SELECT rate FROM fx_rates WHERE from_currency = ? AND to_currency = ? ORDER BY date DESC LIMIT 1"
            cursor = conn.cursor()
            cursor.execute(query, (from_currency, to_currency))
            result = cursor.fetchone()
            if result:
                return result[0]
            # Try inverse conversion if direct rate not found
            cursor.execute(query, (to_currency, from_currency))
            inverse_result = cursor.fetchone()
            if inverse_result:
                return 1 / inverse_result[0]
            raise ValueError(f"FX rate for {from_currency}->{to_currency} not found in database.")
    except sqlite3.Error as e:
        raise ValueError(f"DB error fetching FX rate: {e}. Ensure 'fx_rates' table exists and is populated.")

def add_technical_indicators(df):
    df = df.copy()
    df['sma_14'] = SMAIndicator(df['close'], window=14).sma_indicator()
    df['rsi_14'] = RSIIndicator(df['close'], window=14).rsi()
    df.fillna(method="bfill", inplace=True)
    return df

def arima_forecast(df, periods, order=(5,1,0)):
    model = ARIMA(df['close'], order=order)
    model_fit = model.fit()
    return model_fit.forecast(steps=periods)

def backtest_arima(df, test_size=30, order=(5,1,0)):
    if len(df) <= test_size:
        print(f"Warning: Not enough data for backtesting (data size: {len(df)}, test size: {test_size}). Skipping.")
        return
    train = df['close'][:-test_size]
    test = df['close'][-test_size:]
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=test_size)
    mape = np.mean(np.abs((test.values - forecast.values) / test.values)) * 100
    rmse = np.sqrt(np.mean((test.values - forecast.values) ** 2))
    print(f"Backtest Accuracy (last {test_size} days) -> MAPE: {mape:.2f}%, RMSE: {rmse:.2f}")

def generate_ohlc_from_close(forecast):
    ohlc_data = []
    for i, close_price in enumerate(forecast):
        open_price = forecast.iloc[i-1] if i > 0 else close_price * (1 - 0.01)
        high = max(open_price, close_price) * (1 + np.random.uniform(0.001, 0.01))
        low = min(open_price, close_price) * (1 - np.random.uniform(0.001, 0.01))
        ohlc_data.append({'open': open_price, 'high': high, 'low': low, 'close': close_price})
    return pd.DataFrame(ohlc_data)

def create_forecast_plot(ohlc_forecast, symbol, days, currency):
    \"\"\"Generates an interactive Plotly candlestick chart for the forecast.\"\"\"
    fig = go.Figure(data=[go.Candlestick(
        x=ohlc_forecast.index, open=ohlc_forecast['open'], high=ohlc_forecast['high'],
        low=ohlc_forecast['low'], close=ohlc_forecast['close'],
        increasing_line_color='limegreen', decreasing_line_color='crimson',
        name='Forecast'
    )])
    fig.update_layout(
        title=f"{symbol} Stock Price Forecast ({days} Days) in {currency}",
        xaxis_title='Date', yaxis_title=f'Price ({currency})',
        xaxis_rangeslider_visible=False, plot_bgcolor='white',
        font=dict(family='Arial', size=14)
    )
    return fig

def generate_forecasts(db_path, tickers, forecast_horizon, target_currency):
    \"\"\"
    Generates and displays forecasts for a list of tickers using data from the database.
    \"\"\"
    all_results = {}
    print("\\n" + "="*50)
    print("STARTING FORECASTING PROCESS")
    print("="*50)
    for symbol in tickers:
        try:
            print(f"\\n--- Forecasting for {symbol} ---")
            data = load_data_from_db(db_path, symbol)
            data_with_indicators = add_technical_indicators(data)
            backtest_arima(data_with_indicators, test_size=30, order=(5,1,0))
            print(f"Generating {forecast_horizon}-day forecast...")
            forecast = arima_forecast(data_with_indicators, forecast_horizon, order=(5,1,0))
            native_currency = STOCK_CURRENCY.get(symbol, 'USD')
            display_currency = target_currency
            if native_currency != display_currency:
                try:
                    fx_rate = get_fx_rate_from_db(db_path, native_currency, display_currency)
                    forecast *= fx_rate
                    print(f"Converted forecast from {native_currency} to {display_currency} using rate: {fx_rate:.4f}")
                except ValueError as e:
                    print(f"⚠️ Warning: Could not convert currency. {e}. Displaying in native currency ({native_currency}).")
                    display_currency = native_currency
            ohlc_forecast = generate_ohlc_from_close(forecast)
            future_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=forecast_horizon)
            ohlc_forecast.index = future_dates
            fig = create_forecast_plot(ohlc_forecast, symbol, forecast_horizon, display_currency)
            fig.show()
            all_results[symbol] = {'message': 'Success', 'forecast': ohlc_forecast}
        except Exception as e:
            print(f"❌ Error forecasting for {symbol}: {e}")
            all_results[symbol] = {'message': f'Failed: {e}', 'forecast': None}
    return all_results
"""

with open("stock_forecast.py", "w") as f:
    f.write(stock_forecast_code)

print("✅ 'stock_forecast.py' created successfully.")


✅ 'stock_forecast.py' created successfully.


In [ ]:
import yfinance as yf
import pandas as pd
import sqlite3
import os
from datetime import datetime, timedelta

def populate_database(db_path, tickers, start_date='2020-01-01'):
    """
    Fetches historical market data and FX rates using yfinance
    and populates an SQLite database.
    """
    # Delete the old database file if it exists to ensure a fresh start
    if os.path.exists(db_path):
        os.remove(db_path)
        print(f"Removed existing database at {db_path}")

    print("\n--- Starting Data Population (ETL) ---")
    with sqlite3.connect(db_path) as conn:
        # 1. Fetch and store market data
        for ticker in tickers:
            print(f"Fetching market data for {ticker}...")
            try:
                data = yf.download(ticker, start=start_date, progress=False)

                # --- FIX ADDED HERE ---
                # This check handles cases where yfinance returns a multi-level column index,
                # which can happen even for single tickers. We flatten it to a single level.
                if isinstance(data.columns, pd.MultiIndex):
                    data.columns = data.columns.droplevel(1)
                # --- END OF FIX ---

                if not data.empty:
                    data.reset_index(inplace=True)
                    data.rename(columns={
                        'Date': 'date',
                        'Open': 'open',
                        'High': 'high',
                        'Low': 'low',
                        'Close': 'close',
                        'Volume': 'volume'
                    }, inplace=True)
                    data['symbol'] = ticker
                    data['date'] = data['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
                    data[['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']].to_sql(
                        'market_data', conn, if_exists='append', index=False
                    )
                    print(f"Successfully stored {len(data)} records for {ticker}.")
                else:
                    print(f"⚠️ No data found for {ticker}.")
            except Exception as e:
                print(f"❌ Could not fetch or store data for {ticker}: {e}")

        # 2. Fetch and store FX rates (INR to USD)
        print("\nFetching FX rates (INR to USD)...")
        try:
            fx_data = yf.download('INRUSD=X', start=start_date, progress=False)
            if not fx_data.empty:
                fx_df = fx_data[['Close']].reset_index()
                fx_df.rename(columns={'Date': 'date', 'Close': 'rate'}, inplace=True)
                fx_df['from_currency'] = 'INR'
                fx_df['to_currency'] = 'USD'
                fx_df['date'] = fx_df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
                fx_df.to_sql('fx_rates', conn, if_exists='append', index=False)
                print("Successfully stored FX rates for INR-USD.")
            else:
                print("⚠️ No FX data found for INRUSD=X.")
        except Exception as e:
            print(f"❌ Could not fetch or store FX rates: {e}")

    print("\n--- Data Population Complete ---")

# --- Configuration ---
db_path = "/content/financial_data.db"
tickers_to_process = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'RELIANCE.NS', 'TCS.NS', 'INFY.NS', 'HDFCBANK.NS', 'NVDA']

# --- Run the ETL Process ---
populate_database(db_path, tickers_to_process)


Removed existing database at /content/financial_data.db

--- Starting Data Population (ETL) ---
Fetching market data for AAPL...
Successfully stored 1402 records for AAPL.
Fetching market data for MSFT...
Successfully stored 1402 records for MSFT.
Fetching market data for GOOGL...
Successfully stored 1402 records for GOOGL.
Fetching market data for TSLA...
Successfully stored 1402 records for TSLA.
Fetching market data for RELIANCE.NS...
Successfully stored 1385 records for RELIANCE.NS.
Fetching market data for TCS.NS...
Successfully stored 1385 records for TCS.NS.
Fetching market data for INFY.NS...
Successfully stored 1385 records for INFY.NS.
Fetching market data for HDFCBANK.NS...
Successfully stored 1385 records for HDFCBANK.NS.
Fetching market data for NVDA...
Successfully stored 1402 records for NVDA.

Fetching FX rates (INR to USD)...
Successfully stored FX rates for INR-USD.

--- Data Population Complete ---


In [ ]:
from stock_forecast import generate_forecasts

# --- Interactive Forecasting Configuration ---
db_path = "/content/financial_data.db"
print("--- Interactive Stock Forecaster ---")

# 1. Get Ticker Symbols from User
tickers_input = input("Enter ticker symbols separated by commas (e.g., AAPL, GOOGL, RELIANCE.NS): ")

if not tickers_input:
    print("\n❌ No tickers entered. Halting execution.")
else:
    # Clean up the user input into a list
    tickers_to_forecast = [ticker.strip().upper() for ticker in tickers_input.split(',')]

    # 2. Get Forecast Horizon from User
    try:
        horizon_input = input("Enter the number of days to forecast (e.g., 30): ")
        forecast_horizon = int(horizon_input)
    except (ValueError, TypeError):
        print("⚠️ Invalid number. Defaulting to 30 days.")
        forecast_horizon = 30

    # 3. Get Target Currency from User
    currency_input = input("Enter the target currency (e.g., USD, INR): ")
    target_currency = currency_input.strip().upper() if currency_input else 'USD'
    if not currency_input:
        print("⚠️ No currency entered. Defaulting to USD.")

    # --- Run the Forecasting Process ---
    forecast_results = generate_forecasts(
        db_path=db_path,
        tickers=tickers_to_forecast,
        forecast_horizon=forecast_horizon,
        target_currency=target_currency
    )

    # --- Print Final Summary ---
    print("\n" + "="*50)
    print("FORECASTING SUMMARY")
    print("="*50)
    for symbol, result in forecast_results.items():
        print(f"Ticker: {symbol}")
        print(f"Status: {result['message']}")
        if result['forecast'] is not None:
            print("Forecasted Prices (OHLC):")
            # Print the last 5 days of the forecast
            print(result['forecast'].round(2).tail())
            print("-" * 25)


--- Interactive Stock Forecaster ---
Enter ticker symbols separated by commas (e.g., AAPL, GOOGL, RELIANCE.NS): TSLA
Enter the number of days to forecast (e.g., 30): 33
Enter the target currency (e.g., USD, INR): USD

STARTING FORECASTING PROCESS

--- Forecasting for TSLA ---
Backtest Accuracy (last 30 days) -> MAPE: 3.06%, RMSE: 12.57
Generating 33-day forecast...



FORECASTING SUMMARY
Ticker: TSLA
Status: Success
Forecasted Prices (OHLC):
              open    high     low   close
2025-08-29  308.51  309.78  307.88  308.51
2025-08-30  308.51  310.77  307.00  308.51
2025-08-31  308.51  310.46  306.17  308.51
2025-09-01  308.51  310.01  306.45  308.51
2025-09-02  308.51  309.46  308.20  308.51
-------------------------
